# Imports

In [ ]:
%env OE_LICENSE=/Users/alexpayne/oe_license.txt

In [ ]:
from openeye import oechem
from openeye import oedepict
from sklearn.cluster import KMeans
import numpy as np
import importlib
from covid_moonshot_ml.docking import docking as d
from covid_moonshot_ml.data import openeye as oe
from covid_moonshot_ml import schema
from covid_moonshot_ml.datasets import utils

In [ ]:
importlib.reload(d)
importlib.reload(schema)
importlib.reload(utils)

# Exploring kmeans clustering

In [ ]:
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.predict([[0,0], [10,0]])

In [ ]:
kmeans.cluster_centers_

# Load in MCSS pickle

In [ ]:
import pickle as pkl

In [ ]:
pickle_path = "/Volumes/Rohirrim/local_test/chemiinformatics/mcs_sort_index.pkl"

In [ ]:
with open(pickle_path, 'rb') as f:
    data = pkl.load(f)

In [ ]:
len(data[0])

In [ ]:
len(data[1])

In [ ]:
len(data[2])

In [ ]:
len(data[2][0])

In [ ]:
582-220

In [ ]:
data[2][0]

In [ ]:
data[2][110]

In [ ]:
X = data[2]

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

In [ ]:
cluster_1 = np.array(data[0])[kmeans.labels_==1]

In [ ]:
cmpd_to_label_dict = {data[0][i]: kmeans.labels_[i] for i in range(len(data[0]))}

In [ ]:
cmpd_to_label_dict

In [ ]:
cluster_0 = np.array(data[0])[kmeans.labels_==0]

## get smiles for each molecule

In [ ]:
cmpd_tracker = "/Users/alexpayne/Scientific_Projects/mers-drug-discovery/Mpro_compound_tracker_csv.csv"
xtal_compounds = d.parse_xtal(cmpd_tracker, "")

In [ ]:
len(xtal_compounds)

## examine compound tracker

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(cmpd_tracker)

In [ ]:
df.index = df["Compound ID"]

In [ ]:
df.loc['ALP-POS-c3a96089-4', "Moonshot Series"]

## load cmpd_to_frag

In [ ]:
import yaml

In [ ]:
with open("../data/cmpd_to_frag.yaml") as f:
    cmpd_to_frag_dict = yaml.safe_load(f)

In [ ]:
frag_to_cmpd = {frag: cmpd for cmpd, frag in cmpd_to_frag_dict.items()}

In [ ]:
for xtal in xtal_compounds:
    xtal.compound_id=frag_to_cmpd[xtal.dataset]
    xtal.series=df.loc[xtal.compound_id, "Moonshot Series"]

In [ ]:
len(xtal_compounds)

In [ ]:
frag_data = utils.parse_fragalysis_data(cmpd_tracker, "", xtals_only=False)

In [ ]:
df.loc["AGN-NEW-891393a6-1"]

In [ ]:
set(df["Moonshot Series"])

In [ ]:
cmpd_info = [frag_data.get(cmpd_id) for cmpd_id in data[0]]

In [ ]:
sum(np.array(cmpd_info) != None)

In [ ]:
cmpd_list = []
for cmpd_id in data[0]:
    try:
        cmpd_data = frag_data.get(cmpd_id)
        if cmpd_data:
            moonshot_series_name = df.loc[cmpd_id, "Moonshot Series"]
            if type(moonshot_series_name) != str:
                moonshot_series_name = moonshot_series_name[0]
            if moonshot_series_name == "None":
                moonshot_series_name = None 
            cmpd_data.series = moonshot_series_name
            cmpd_list.append(cmpd_data)
    except KeyError:
        continue

In [ ]:
cmpd_list[0]

In [ ]:
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, cmpd_list[0].smiles)
oe.write_openeye_ligand(mol, "test.png")

In [ ]:
mapped_data = {}
for i in range(len(data[0])):
    cmpd_data = cmpd_info[i]
    if cmpd_data:
        mapped_data[cmpd_data.compound_id] = (cmpd_data, data[2][i])

In [ ]:
len(mapped_data)

In [ ]:
mapped_data["AAR-POS-5507155c-1"]

In [ ]:
key_filter = [key for key in frag_data.keys() if "BEN-DND" in key]

In [ ]:
key_filter.sort()
key_filter

In [ ]:
data[0]

## write out first 10 structures for each cluster

In [ ]:
from covid_moonshot_ml.data import openeye as oe

In [ ]:
cluster = 7
for i in range(10):
    cmpd_id = np.array(data[0])[kmeans.labels_ == cluster][i]
    cmpd_data = mapped_data.get(cmpd_id)
    if cmpd_data:
        smiles = cmpd_data[0].smiles
        mol = oechem.OEGraphMol()
        oechem.OESmilesToMol(mol, smiles)
        oe.write_openeye_ligand(mol, 
                            out_fn=f"/Volumes/Rohirrim/local_test/chemiinformatics/cluster{cluster}_{cmpd_id}.png")

# Play with options for MCSS based clustering

In [ ]:
cmpd_id = np.array(data[0])[kmeans.labels_ == cluster][0]
smiles = mapped_data[cmpd_id][0].smiles
mol1 = oechem.OEGraphMol()
oechem.OESmilesToMol(mol1, smiles)

## Generate molecule fingerprints

In [ ]:
from openeye import oegraphsim

In [ ]:
mol = oechem.OEGraphMol()
oechem.OESmilesToMol(mol, cmpd_list[0].smiles)
fp = oegraphsim.OEFingerPrint()
oegraphsim.OEMakeFP(fp, mol, oegraphsim.OEFPType_Circular)

In [ ]:
fptype = oegraphsim.OEGetFPType(oegraphsim.OEFPType_Circular)

In [ ]:
fptype.GetFPTypeString()

In [ ]:
fp.IsValid()

In [ ]:
fps = []
for cmpd_data in cmpd_list:
    mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(mol, cmpd_data.smiles)
    fp = oegraphsim.OEFingerPrint()
    oegraphsim.OEMakeFP(fp, mol, oegraphsim.OEFPType_Circular)
    if fp.IsValid():
        fps.append(fp)
    
    else:
        print(cmpd_data.compound_id)
fps_array = np.array(fps)

In [ ]:
fp.GetSize()

In [ ]:
oebit = oechem.OEBitVector()
oechem.OEParseHex(oebit, fp.ToHexString())

In [ ]:
{fp.GetSize() for fp in fps}

In [ ]:
fp = fps[1]
print(fp.GetSize())

In [ ]:
integer = int(hex_str[:-1], base=16)

In [ ]:
len(str(integer))

In [ ]:
binary_str = f"{integer:04096b}"

In [ ]:
len(binary_str)

In [ ]:
vector_array = np.array(list(map(int, binary_str)))

In [ ]:
vector_array

In [ ]:
fp.GetSize()

In [ ]:
4096 / 

In [ ]:
smiles = ['']

## actually generate vector arrays

In [ ]:
def fp_to_vector_array(fp):
    integer = int(fp.ToHexString()[:-1], base=16)
    binary_str = f"{integer:04096b}"
    vector_array = np.array(list(map(int, binary_str)))
    return vector_array

In [ ]:
vector_arrays = [fp_to_vector_array(fp) for fp in fps]

In [ ]:
len(vector_arrays)

## Try clustering with fprints

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(vector_arrays)

In [ ]:
cmpd_ids = np.array([xtal.compound_id for xtal in cmpd_list])
cmpd_series = np.array([xtal.series for xtal in cmpd_list])

In [ ]:
results_df = pd.DataFrame({"Compound_ID":cmpd_ids, "Series":cmpd_series, "Cluster":kmeans.labels_})

In [ ]:
results_df[results_df.Cluster == 9]

In [ ]:
set(results_df[results_df.Cluster == 0].Series)

In [ ]:
set(results_df[results_df.Cluster == ].Series)

In [ ]:
for cluster in results_df.Cluster:
    filtered_cmpd_ids = np.array(cmpd_ids)[kmeans.labels_ == cluster]
    for i in range(len(filtered_cmpd_ids)):
        cmpd_id = filtered_cmpd_ids[i]
        cmpd_data = mapped_data.get(cmpd_id)
        if cmpd_data:
            smiles = cmpd_data[0].smiles
            mol = oechem.OEGraphMol()
            oechem.OESmilesToMol(mol, smiles)
            oe.write_openeye_ligand(mol, 
                                out_fn=f"/Volumes/Rohirrim/local_test/chemiinformatics/cluster{cluster}_{cmpd_id}.png")